## Airline Crew Scheduling


### Airline Crew scheduling helps to optimally schedule the crew work shifts considering equal distribution of workloads among the crew members.

This sample notebook shows you how to deploy Airline Crew Scheduling using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Airline Crew Scheduling. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page Product Review Aspect Detection: Routers. 
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn='arn:aws:sagemaker:us-east-2:786796469737:model-package/crew-sch'

In [2]:
import base64
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from urllib.parse import urlparse
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
import urllib.request
import numpy as np

In [3]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-2-786796469737'

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name='crew-sch'

content_type='application/zip'

real_time_inference_instance_type='ml.m5.large'
batch_transform_inference_instance_type='ml.m5.large'

#### A. Create an endpoint

In [5]:

def predict_wrapper(endpoint, session):
    return sage.predictor.Predictor(endpoint, session,content_type)

#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

#Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

----!

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

In [6]:
file_name = 'Input/Input.zip'

<Add code snippet that shows the payload contents>

#### C. Perform real-time inference

In [7]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $model_name \
    --body fileb://$file_name \
    --content-type $content_type \
    --region $sagemaker_session.boto_region_name \
    output.txt

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Visualize output

In [8]:
import json
with open('output.txt', 'r') as f:
    output = json.load(f)
print(output)

{'BASE1': {'c1': ['p190', 'p110', 'p258', 'p141', 'p180', 'p9', 'p210', 'p227', 'p245', 'p3', 'p181', 'p34', 'p285', 'p294', 'p75', 'p155'], 'c10': ['p40', 'p217', 'p108', 'p142', 'p203', 'p100', 'p279', 'p226', 'p248', 'p11', 'p254', 'p2', 'p250', 'p290', 'p287', 'p295', 'p74', 'p156'], 'c2': ['p189', 'p42', 'p143', 'p139', 'p99', 'p179', 'p278', 'p25', 'p209', 'p225', 'p246', 'p230', 'p12', 'p253', 'p94', 'p249', 'p284', 'p154'], 'c3': ['p188', 'p218', 'p109', 'p140', 'p204', 'p8', 'p288', 'p26', 'p211', 'p231', 'p255', 'p4', 'p182', 'p93', 'p251', 'p33', 'p286', 'p293', 'p73', 'p153'], 'c4': ['p188', 'p218', 'p109', 'p140', 'p204', 'p8', 'p288', 'p26', 'p211', 'p231', 'p255', 'p4', 'p182', 'p93', 'p251', 'p33', 'p286', 'p293', 'p73', 'p153'], 'c5': ['p190', 'p110', 'p258', 'p141', 'p180', 'p9', 'p210', 'p227', 'p245', 'p3', 'p181', 'p34', 'p285', 'p294', 'p75', 'p155'], 'c6': ['p187', 'p41', 'p219', 'p138', 'p202', 'p98', 'p178', 'p7', 'p10', 'p27', 'p247', 'p232', 'p252', 'p183', '

#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [9]:
predictor=sage.predictor.Predictor(model_name, sagemaker_session,content_type)
predictor.delete_endpoint(delete_endpoint_config=True)

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [10]:
#upload the batch-transform job input files to S3
transform_input_folder = "Input"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name) 
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-786796469737/crew-sch


In [11]:

#Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

.................... * Serving Flask app 'serve' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on all addresses.
 * Running on http://169.254.255.131:8080/ (Press CTRL+C to quit)
169.254.255.130 - - [10/Feb/2022 09:38:44] "GET /ping HTTP/1.1" 200 -
169.254.255.130 - - [10/Feb/2022 09:38:44] "#033GET /execution-parameters HTTP/1.1#033" 404 -
2022-02-10T09:38:44.057:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
All the flights are assigned for BASE1
All the flights are assigned for BASE1
All the flights are assigned for BASE2
All the flights are assigned for BASE2
All the flights are assigned for BASE3
Final assignment of crew for BASE1
####################################
{'c8': ['p187', 'p41', 'p219', 'p138', 'p202', 'p98', 'p178', 'p7', 'p10', 'p27', 'p247', 'p232', 'p252', 'p183', 'p95', 'p289', 'p296', 'p72'], 'c6': ['p187', 'p41', 'p219', 'p138', 'p202', 'p98', 'p178', 'p7

In [15]:
import os
s3_conn = boto3.client("s3")
with open('output.txt', 'wb') as f:
    s3_conn.download_fileobj(bucket, os.path.basename(transformer.output_path)+'/Input.zip.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


In [16]:
import sagemaker

In [17]:
ses = sagemaker.Session()
buck=ses.default_bucket()
print(buck)

sagemaker-us-east-2-786796469737


In [19]:
with open('output.txt', 'r') as f:
    output = json.load(f)
print(output)

{'BASE1': {'c1': ['p190', 'p110', 'p258', 'p141', 'p180', 'p9', 'p210', 'p227', 'p245', 'p3', 'p181', 'p34', 'p285', 'p294', 'p75', 'p155'], 'c10': ['p40', 'p217', 'p108', 'p142', 'p203', 'p100', 'p279', 'p226', 'p248', 'p11', 'p254', 'p2', 'p250', 'p290', 'p287', 'p295', 'p74', 'p156'], 'c2': ['p189', 'p42', 'p143', 'p139', 'p99', 'p179', 'p278', 'p25', 'p209', 'p225', 'p246', 'p230', 'p12', 'p253', 'p94', 'p249', 'p284', 'p154'], 'c3': ['p188', 'p218', 'p109', 'p140', 'p204', 'p8', 'p288', 'p26', 'p211', 'p231', 'p255', 'p4', 'p182', 'p93', 'p251', 'p33', 'p286', 'p293', 'p73', 'p153'], 'c4': ['p188', 'p218', 'p109', 'p140', 'p204', 'p8', 'p288', 'p26', 'p211', 'p231', 'p255', 'p4', 'p182', 'p93', 'p251', 'p33', 'p286', 'p293', 'p73', 'p153'], 'c5': ['p190', 'p110', 'p258', 'p141', 'p180', 'p9', 'p210', 'p227', 'p245', 'p3', 'p181', 'p34', 'p285', 'p294', 'p75', 'p155'], 'c6': ['p187', 'p41', 'p219', 'p138', 'p202', 'p98', 'p178', 'p7', 'p10', 'p27', 'p247', 'p232', 'p252', 'p183', '

### 4. Clean-up

#### A. Delete the model

In [20]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

